In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD,Adadelta,Adam
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg16 import VGG16,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))


nb_train_samples = 20000
nb_val_samples = 2424
image_size = (224,224)
batch_size = 128
class_num = 10

In [3]:
def image_preprocess(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    return img

In [4]:
def test_model(model,test_path,csv='sample_submission.csv'):
    columns = ['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    df = pd.DataFrame(columns=columns)
    
    file_list = os.listdir(test_path)
    for i,file in enumerate(file_list):
        test_data = image_preprocess(test_path + '/' + file)
        y_pred = model.predict(test_data,batch_size=1,verbose=0)
        y_pred = np.clip(y_pred,0.001,0.999)
        y_pred = y_pred[0].tolist()
        df.loc[i] = [file] + y_pred
        
    df.to_csv(csv,index=None)

In [5]:
train_datagen = image.ImageDataGenerator(
                                         shear_range=0.2,
                                         zoom_range=0.2,
                                         rotation_range=15,
                                         height_shift_range=0.2,
                                         width_shift_range=0.2,
                                         channel_shift_range=10,
                                         preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


In [6]:
vgg16_base_model = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))

x = vgg16_base_model.output

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predict = Dense(10, activation='softmax')(x)

vgg16_model = Model(inputs=vgg16_base_model.input, outputs=predict)

In [7]:
for layer in vgg16_model.layers[:19]:
    layer.trainable = False

for i,layer in enumerate(vgg16_model.layers):
    print(i,layer.name,layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False
19 flatten_1 True
20 dense_1 True
21 batch_normalization_1 True
22 dropout_1 True
23 dense_2 True
24 batch_normalization_2 True
25 dropout_2 True
26 dense_3 True


In [8]:
vgg16_model.compile(loss='categorical_crossentropy',
                    #optimizer=optimizers.SGD(lr=1e-3, momentum=0.9) #,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(lr=1e-3),
#                     optimizer=Adadelta(),
                    metrics=['accuracy'])

# fine-tune the model
vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=15,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[EarlyStopping(patience=3)])

vgg16_model.save('vgg16_model_fc.h5')

Epoch 1/15
156/156 [==============================] - 322s 2s/step - loss: 1.9653 - acc: 0.3853 - val_loss: 1.3089 - val_acc: 0.6102
Epoch 2/15
156/156 [==============================] - 325s 2s/step - loss: 0.8922 - acc: 0.6984 - val_loss: 1.0660 - val_acc: 0.6742
Epoch 3/15
156/156 [==============================] - 323s 2s/step - loss: 0.6313 - acc: 0.7896 - val_loss: 1.5527 - val_acc: 0.5932
Epoch 4/15
156/156 [==============================] - 321s 2s/step - loss: 0.5114 - acc: 0.8282 - val_loss: 1.6170 - val_acc: 0.5967
Epoch 5/15
156/156 [==============================] - 318s 2s/step - loss: 0.4352 - acc: 0.8572 - val_loss: 1.4819 - val_acc: 0.5980


In [9]:
vgg16_model = load_model('vgg16_model_fc.h5')

for layer in vgg16_model.layers[:15]:
    layer.trainable = False
    
for layer in vgg16_model.layers[15:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),
                    optimizer=optimizers.Adam(lr=1e-3),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=3)])

vgg16_model.save('vgg16_model_15.h5')

Epoch 1/20
156/156 [==============================] - 325s 2s/step - loss: 0.5677 - acc: 0.8278 - val_loss: 1.1804 - val_acc: 0.7626
Epoch 2/20
156/156 [==============================] - 325s 2s/step - loss: 0.1453 - acc: 0.9557 - val_loss: 0.5371 - val_acc: 0.8724
Epoch 3/20
156/156 [==============================] - 322s 2s/step - loss: 0.0846 - acc: 0.9756 - val_loss: 0.8030 - val_acc: 0.8219
Epoch 4/20
156/156 [==============================] - 321s 2s/step - loss: 0.0728 - acc: 0.9776 - val_loss: 0.9664 - val_acc: 0.7474
Epoch 5/20
156/156 [==============================] - 322s 2s/step - loss: 0.0559 - acc: 0.9828 - val_loss: 1.0667 - val_acc: 0.7770


In [10]:
vgg16_model = load_model('vgg16_model_15.h5')

for layer in vgg16_model.layers[:11]:
    layer.trainable = False
    
for layer in vgg16_model.layers[11:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                     optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=3)])

vgg16_model.save('vgg16_model_11.h5')

Epoch 1/20
156/156 [==============================] - 323s 2s/step - loss: 0.0682 - acc: 0.9789 - val_loss: 1.0124 - val_acc: 0.7300
Epoch 2/20
156/156 [==============================] - 327s 2s/step - loss: 0.0481 - acc: 0.9850 - val_loss: 0.4275 - val_acc: 0.8855
Epoch 3/20
156/156 [==============================] - 324s 2s/step - loss: 0.0264 - acc: 0.9920 - val_loss: 0.4779 - val_acc: 0.8449
Epoch 4/20
156/156 [==============================] - 321s 2s/step - loss: 0.0230 - acc: 0.9940 - val_loss: 0.5554 - val_acc: 0.8463
Epoch 5/20
156/156 [==============================] - 318s 2s/step - loss: 0.0212 - acc: 0.9934 - val_loss: 1.3124 - val_acc: 0.6572


In [11]:
vgg16_model = load_model('vgg16_model_11.h5')

for layer in vgg16_model.layers[:7]:
    layer.trainable = False
    
for layer in vgg16_model.layers[7:]:
    layer.trainable = True

vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
                    optimizer=optimizers.Adam(lr=5e-5),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=3)])
vgg16_model.save('vgg16_model_7.h5')

Epoch 1/20
156/156 [==============================] - 321s 2s/step - loss: 0.0204 - acc: 0.9936 - val_loss: 0.6218 - val_acc: 0.8307
Epoch 2/20
156/156 [==============================] - 326s 2s/step - loss: 0.0144 - acc: 0.9961 - val_loss: 0.6674 - val_acc: 0.8367
Epoch 3/20
156/156 [==============================] - 323s 2s/step - loss: 0.0139 - acc: 0.9964 - val_loss: 0.9728 - val_acc: 0.7644
Epoch 4/20
156/156 [==============================] - 321s 2s/step - loss: 0.0154 - acc: 0.9958 - val_loss: 0.3777 - val_acc: 0.8850
Epoch 5/20
156/156 [==============================] - 319s 2s/step - loss: 0.0101 - acc: 0.9971 - val_loss: 0.5748 - val_acc: 0.8393
Epoch 6/20
156/156 [==============================] - 317s 2s/step - loss: 0.0112 - acc: 0.9967 - val_loss: 1.0363 - val_acc: 0.7204
Epoch 7/20
156/156 [==============================] - 317s 2s/step - loss: 0.0121 - acc: 0.9960 - val_loss: 0.7628 - val_acc: 0.8127


In [12]:
vgg16_model = load_model('vgg16_model_7.h5')

# for layer in vgg16_model.layers[:7]:
#     layer.trainable = False
    
for layer in vgg16_model.layers:
    layer.trainable = True

vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
                    optimizer=optimizers.Adam(lr=5e-5),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=15,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=3)])

vgg16_model.save('vgg16_model_all.h5')

Epoch 1/15
156/156 [==============================] - 321s 2s/step - loss: 0.0128 - acc: 0.9959 - val_loss: 0.3994 - val_acc: 0.8997
Epoch 2/15
156/156 [==============================] - 326s 2s/step - loss: 0.0125 - acc: 0.9965 - val_loss: 0.4805 - val_acc: 0.8924
Epoch 3/15
156/156 [==============================] - 323s 2s/step - loss: 0.0111 - acc: 0.9962 - val_loss: 1.4662 - val_acc: 0.6646
Epoch 4/15
156/156 [==============================] - 321s 2s/step - loss: 0.0084 - acc: 0.9981 - val_loss: 0.5677 - val_acc: 0.8637


In [13]:
vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
                    optimizer=optimizers.Adam(lr=1e-5),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=15,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=5)])

Epoch 1/15
156/156 [==============================] - 322s 2s/step - loss: 0.0028 - acc: 0.9995 - val_loss: 0.3332 - val_acc: 0.9240
Epoch 2/15
156/156 [==============================] - 326s 2s/step - loss: 0.0024 - acc: 0.9995 - val_loss: 0.4448 - val_acc: 0.9016
Epoch 3/15
156/156 [==============================] - 324s 2s/step - loss: 0.0028 - acc: 0.9994 - val_loss: 0.5931 - val_acc: 0.8593
Epoch 4/15
156/156 [==============================] - 322s 2s/step - loss: 0.0015 - acc: 0.9998 - val_loss: 0.4288 - val_acc: 0.8859
Epoch 5/15
156/156 [==============================] - 319s 2s/step - loss: 0.0014 - acc: 0.9997 - val_loss: 0.6277 - val_acc: 0.8528
Epoch 6/15
156/156 [==============================] - 319s 2s/step - loss: 0.0017 - acc: 0.9996 - val_loss: 0.4019 - val_acc: 0.8933


In [14]:
vgg16_model.save('vgg16_final_model.h5')

In [15]:
vgg16_model = load_model('vgg16_final_model.h5')

test_model(vgg16_model,test_path,csv='vgg16_submission_0430.csv')

In [6]:
vgg16_model = load_model('vgg16_model_15.h5')

for layer in vgg16_model.layers[:11]:
    layer.trainable = False
    
for layer in vgg16_model.layers[11:]:
    layer.trainable = True

vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
                    optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=5)])
vgg16_model.save('vgg16_model_11_0417.h5')

Epoch 1/20
156/156 [==============================] - 350s 2s/step - loss: 0.0627 - acc: 0.9815 - val_loss: 0.3219 - val_acc: 0.9214
Epoch 2/20
156/156 [==============================] - 353s 2s/step - loss: 0.0334 - acc: 0.9896 - val_loss: 0.2629 - val_acc: 0.9325
Epoch 3/20
156/156 [==============================] - 343s 2s/step - loss: 0.0332 - acc: 0.9901 - val_loss: 1.0057 - val_acc: 0.7247
Epoch 4/20
156/156 [==============================] - 344s 2s/step - loss: 0.0241 - acc: 0.9930 - val_loss: 1.0485 - val_acc: 0.7252
Epoch 5/20
156/156 [==============================] - 345s 2s/step - loss: 0.0473 - acc: 0.9859 - val_loss: 0.5932 - val_acc: 0.8676
Epoch 6/20
156/156 [==============================] - 345s 2s/step - loss: 0.0291 - acc: 0.9908 - val_loss: 0.3177 - val_acc: 0.9242
Epoch 7/20
156/156 [==============================] - 344s 2s/step - loss: 0.0161 - acc: 0.9951 - val_loss: 0.9984 - val_acc: 0.7988


In [7]:
vgg16_model = load_model('vgg16_model_11_0417.h5')

for layer in vgg16_model.layers[:11]:
    layer.trainable = False
    
for layer in vgg16_model.layers[11:]:
    layer.trainable = True

vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
                    optimizer=optimizers.Adam(lr=5e-5),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=3)])
vgg16_model.save('vgg16_model_11_0427.h5')

Epoch 1/20
156/156 [==============================] - 530s 3s/step - loss: 0.0148 - acc: 0.9958 - val_loss: 1.2274 - val_acc: 0.6858
Epoch 2/20
156/156 [==============================] - 355s 2s/step - loss: 0.0104 - acc: 0.9975 - val_loss: 0.3978 - val_acc: 0.8911
Epoch 3/20
156/156 [==============================] - 349s 2s/step - loss: 0.0059 - acc: 0.9983 - val_loss: 0.4396 - val_acc: 0.9020
Epoch 4/20
156/156 [==============================] - 350s 2s/step - loss: 0.0065 - acc: 0.9982 - val_loss: 0.8292 - val_acc: 0.8571
Epoch 5/20
156/156 [==============================] - 349s 2s/step - loss: 0.0114 - acc: 0.9962 - val_loss: 0.9681 - val_acc: 0.8358


In [18]:
vgg16_model = load_model('vgg16_model_11_0427.h5')

for layer in vgg16_model.layers[:7]:
    layer.trainable = False
    
for layer in vgg16_model.layers[7:]:
    layer.trainable = True

vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
                    optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=3)])
vgg16_model.save('vgg16_model_7_0427.h5')

Epoch 1/20
156/156 [==============================] - 427s 3s/step - loss: 0.0306 - acc: 0.9904 - val_loss: 2.1839 - val_acc: 0.5339
Epoch 2/20
156/156 [==============================] - 426s 3s/step - loss: 0.0245 - acc: 0.9918 - val_loss: 0.5734 - val_acc: 0.8754
Epoch 3/20
156/156 [==============================] - 423s 3s/step - loss: 0.0308 - acc: 0.9901 - val_loss: 0.4802 - val_acc: 0.8724
Epoch 4/20
156/156 [==============================] - 425s 3s/step - loss: 0.0257 - acc: 0.9923 - val_loss: 0.4923 - val_acc: 0.9042
Epoch 5/20
156/156 [==============================] - 427s 3s/step - loss: 0.0180 - acc: 0.9949 - val_loss: 0.7201 - val_acc: 0.8114
Epoch 6/20
156/156 [==============================] - 426s 3s/step - loss: 0.0158 - acc: 0.9951 - val_loss: 0.5378 - val_acc: 0.8841


In [6]:
vgg16_model = load_model('vgg16_model_7_0427.h5')
vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
                    optimizer=optimizers.Adam(lr=1e-5),
                    metrics=['accuracy'])  
vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune')])

Epoch 1/10
156/156 [==============================] - 543s 3s/step - loss: 0.0075 - acc: 0.9981 - val_loss: 0.4138 - val_acc: 0.9076
Epoch 2/10
156/156 [==============================] - 392s 3s/step - loss: 0.0038 - acc: 0.9989 - val_loss: 0.4365 - val_acc: 0.8907
Epoch 3/10
156/156 [==============================] - 385s 2s/step - loss: 0.0027 - acc: 0.9992 - val_loss: 0.4412 - val_acc: 0.9072
Epoch 4/10
156/156 [==============================] - 385s 2s/step - loss: 0.0023 - acc: 0.9995 - val_loss: 0.4414 - val_acc: 0.9081
Epoch 5/10
156/156 [==============================] - 385s 2s/step - loss: 0.0025 - acc: 0.9995 - val_loss: 0.4536 - val_acc: 0.9046
Epoch 6/10
156/156 [==============================] - 385s 2s/step - loss: 0.0026 - acc: 0.9992 - val_loss: 0.3720 - val_acc: 0.9094
Epoch 7/10
156/156 [==============================] - 385s 2s/step - loss: 0.0024 - acc: 0.9993 - val_loss: 0.5023 - val_acc: 0.8820
Epoch 8/10
156/156 [==============================] - 386s 2s/step - 

In [7]:
vgg16_model.save('vgg16_model_7_0427.h5')

In [7]:
vgg16_model = load_model('vgg16_model_7_0427.h5')

test_model(vgg16_model,test_path,csv='vgg16_submission_7_0427.csv')

In [6]:
vgg16_model = load_model('vgg16_model_7_0427.h5')

    
for layer in vgg16_model.layers:
    layer.trainable = True

vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
                    optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=3)])
vgg16_model.save('vgg16_model_0427_openall.h5')

Epoch 1/20
156/156 [==============================] - 508s 3s/step - loss: 0.0218 - acc: 0.9935 - val_loss: 0.7033 - val_acc: 0.8290
Epoch 2/20
156/156 [==============================] - 325s 2s/step - loss: 0.0212 - acc: 0.9933 - val_loss: 0.5558 - val_acc: 0.8990
Epoch 3/20
156/156 [==============================] - 326s 2s/step - loss: 0.0198 - acc: 0.9936 - val_loss: 0.4090 - val_acc: 0.9059
Epoch 4/20
156/156 [==============================] - 326s 2s/step - loss: 0.0130 - acc: 0.9964 - val_loss: 0.8535 - val_acc: 0.8057
Epoch 5/20
156/156 [==============================] - 321s 2s/step - loss: 0.0129 - acc: 0.9965 - val_loss: 0.6237 - val_acc: 0.8850
Epoch 6/20
156/156 [==============================] - 321s 2s/step - loss: 0.0066 - acc: 0.9979 - val_loss: 0.7064 - val_acc: 0.8349


In [7]:
vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.Adam(lr=1e-5),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=5)])

Epoch 1/20
156/156 [==============================] - 323s 2s/step - loss: 0.0035 - acc: 0.9990 - val_loss: 0.3623 - val_acc: 0.9097
Epoch 2/20
156/156 [==============================] - 327s 2s/step - loss: 0.0022 - acc: 0.9994 - val_loss: 0.4056 - val_acc: 0.9046
Epoch 3/20
156/156 [==============================] - 326s 2s/step - loss: 0.0014 - acc: 0.9996 - val_loss: 0.5085 - val_acc: 0.8868
Epoch 4/20
156/156 [==============================] - 322s 2s/step - loss: 0.0012 - acc: 0.9998 - val_loss: 0.5729 - val_acc: 0.8754
Epoch 5/20
156/156 [==============================] - 321s 2s/step - loss: 0.0015 - acc: 0.9997 - val_loss: 0.4656 - val_acc: 0.8894
Epoch 6/20
156/156 [==============================] - 321s 2s/step - loss: 0.0011 - acc: 0.9998 - val_loss: 0.4607 - val_acc: 0.9064


In [8]:
vgg16_model.save('vgg16_model_0427_openall.h5')